# AEFIT

This would be the first attempt to run the unsupervised learning VAE network to learn how to characterize a 1D profile with atted noise and missing input.

More than a simple fit, this method should learn 

In [14]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh
import Hunch_utils  as Htls
import Hunch_lsplot as Hplt

%aimport Dummy_g1data

import Dummy_g1data as dummy

%aimport models.base
%aimport models.AEFIT
%aimport models.AEFIT5
# ipysh.Bootstrap_support.debug()

## Data and Model
The model and data generator are set:
Dummy data generator generates from a set of 5 kind of curves with a dataset cardinality of 60K samples.

All the shapes are generated from a dictionary array that defines mean sigma and gain of sum of gaussians.
This table is printed from the variable ds.kinds

>NOTE: 
> The actual model is generating random so it is not redoing the very same samples on each epoch.
> To exactly constraint the maximum size of the dataset the tf buffer can be used

the model uses bby default an input of 40 samples that are the (x,y) tuple values of 20 points from the generated shapes.


In [15]:
ds = dummy.Dummy_g1data(counts=10000).buffer()
ds.kinds

[{'gain': [1, 1], 'mean': [0.2, 0.8], 'sigma': [0.1, 0.1]},
 {'gain': [0.5], 'mean': [0.8], 'sigma': [0.1]},
 {'gain': [0.5], 'mean': [0.2], 'sigma': [0.1]},
 {'gain': [1], 'mean': [0.5], 'sigma': [0.2]},
 {'gain': [0.5], 'mean': [0.5], 'sigma': [0.2]}]

In [ ]:
# m = aefit.AEFIT(latent_dim=2)
# aefit.test_dummy(m, data=ds, epoch=5, batch=200, loss_factor=1e-3)

In [25]:
dds = ds.ds_array.map(lambda xy,l: (xy,xy) )
# [x for x in dds.take(1)][0]

In [26]:
m3 = models.AEFIT5.AEFIT5(latent_dim=2, scale=1., beta=0., geometry=[20,20,10,10])

AEFIT5 a ready:


In [27]:
# models.base.train_thread(m3, ds, epoch=5, batch=200, learning_rate=1e-3).control_panel()
h = m3.fit(dds.skip(3000).batch(100, drop_remainder=True), validation_data=dds.take(3000).batch(100), epochs=20, shuffle=False)

Epoch 1/20
70/70 [==============================] - 4s 57ms/step - loss: 0.0583 - val_loss: 0.0000e+00
Epoch 2/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0272 - val_loss: 0.0230
Epoch 3/20
70/70 [==============================] - 3s 38ms/step - loss: 0.0190 - val_loss: 0.0169
Epoch 4/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0136 - val_loss: 0.0126
Epoch 5/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0098 - val_loss: 0.0086
Epoch 6/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0083 - val_loss: 0.0081
Epoch 7/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0079 - val_loss: 0.0081
Epoch 8/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0077 - val_loss: 0.0072
Epoch 9/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0075 - val_loss: 0.0072
Epoch 10/20
70/70 [==============================] - 3s 37ms/step - loss: 0.0072 - val_loss: 0.0069
Epoch

In [28]:
p = Hplt.LSPlotBokeh()
p.set_model(m3)
p.set_data(ds)
p.plot(notebook_url='http://172.17.0.2:8888')

Loading BokehJS ...

In [28]:
relevance = m3.generative_net.layers[0]
relevance.weights

[<tf.Variable 'aefi_t5_5_gRlv/kernel:0' shape=(2,) dtype=float32, numpy=array([0.9999999, 0.9968099], dtype=float32)>]

## Missing data simulation

Now we want to test the network against particular shapes within the latent main paths but with added noise and simulated missing data. The function simulate_missing_data reduce de number of available input simply duplicating the point that precedes the missing one with the same value.

A further gaussian noise has been also applied.

In [ ]:

def simulate_missing_data(m, lpt=[0.5,-1.6], noise_var=0.05, arr = [3,2,1,5,8,7,6,9,12,11,14,13,18]):
    xy = m.decode(tf.convert_to_tensor([pt]),apply_sigmoid=True, training=False)
    x,y = tf.split(xy[0], 2)
    x,y = (x.numpy(), y.numpy())

    fig = plt.figure('gen_missing_curve',figsize=(18, 6))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)    
    
    ax1.set_xlim(-2.,2.)
    ax1.set_ylim(-2.,2.)
    
    ax1.scatter(pt[0],pt[1],s=80)
    ax2.scatter(x,y,s=40)

    # apply noise
    x += np.random.normal(0,noise_var,len(x))
    y += np.random.normal(0,noise_var,len(y))

    # apply missing data simulation
    for i,v in enumerate(arr,0):
        x[arr[i]]=x[arr[i]+1]
        y[arr[i]]=y[arr[i]+1]
    
    ax2.scatter(x,y,s=80)

    me,va = m.encode(tf.reshape(tf.concat([x,y],0), shape=[1,-1]), training=False)
    print("Guessed Latent point = ",me.numpy())
    gpt = me[0].numpy()
    ax1.scatter(gpt[0],gpt[1])
    
    XY = m.decode(me,apply_sigmoid=True, training=False)
    X,Y = tf.split(XY[0], 2)
    X,Y = (X.numpy(), Y.numpy())
    # plt.figure('reconstructed')
    ax2.scatter(X,Y,s=40)
    # plt.plot(X,Y)



We start by generating close to the shape {'mean': [0.5], 'sigma': [0.2], 'gain': [0.5]}

This is in the middle of the central cluster

In [ ]:

    
# generate from point: 0.6, -0.7
pt = [0.6,-0.7]
noise_var = 0.1
arr = [3,2,1,5,8,7,6,9,12,11,14,13,18]
simulate_missing_data(m3, pt,noise_var,arr)


Now we want to check if the nework can simulate a point in the middle od two clusters

In [ ]:
# generate from point: 0.5, -1.6
pt = [0.5,-1.6]
noise_var = 0.05
arr = [3,2,1,5,8,7,6,9,12,11,14,13,18]
simulate_missing_data(m3, pt,noise_var,arr)

